In [1]:
!kubectl get pods

No resources found in default namespace.


In [2]:
!kubectl describe node

Name:               docker-desktop
Roles:              control-plane
Labels:             beta.kubernetes.io/arch=amd64
                    beta.kubernetes.io/os=linux
                    kubernetes.io/arch=amd64
                    kubernetes.io/hostname=docker-desktop
                    kubernetes.io/os=linux
                    node-role.kubernetes.io/control-plane=
                    node.kubernetes.io/exclude-from-external-load-balancers=
Annotations:        kubeadm.alpha.kubernetes.io/cri-socket: unix:///var/run/cri-dockerd.sock
                    node.alpha.kubernetes.io/ttl: 0
                    volumes.kubernetes.io/controller-managed-attach-detach: true
CreationTimestamp:  Thu, 28 Aug 2025 20:11:38 -0400
Taints:             <none>
Unschedulable:      false
Lease:
  HolderIdentity:  docker-desktop
  AcquireTime:     <unset>
  RenewTime:       Sun, 31 Aug 2025 00:36:13 -0400
Conditions:
  Type             Status  LastHeartbeatTime                 LastTransitionTime          

In [3]:
!kubectl apply -f local-volume.yaml

persistentvolume/local-volume created


In [4]:
!kubectl get persistentvolume

NAME           CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS      CLAIM   STORAGECLASS    VOLUMEATTRIBUTESCLASS   REASON   AGE
local-volume   1Gi        RWO            Retain           Available           local-storage   <unset>                          19s


In [5]:
!kubectl describe pv local-volume

Name:              local-volume
Labels:            <none>
Annotations:       <none>
Finalizers:        [kubernetes.io/pv-protection]
StorageClass:      local-storage
Status:            Available
Claim:             
Reclaim Policy:    Retain
Access Modes:      RWO
VolumeMode:        Filesystem
Capacity:          1Gi
Node Affinity:     
  Required Terms:  
    Term 0:        kubernetes.io/hostname in [docker-desktop]
Message:           
Source:
    Type:  LocalVolume (a persistent volume backed by local storage on a node)
    Path:  /mnt/disk/local1
Events:    <none>


In [6]:
!kubectl apply -f local-volume.yaml

persistentvolume/local-volume unchanged
persistentvolumeclaim/local-volume-claim created


In [7]:
!kubectl get pvc

NAME                 STATUS    VOLUME   CAPACITY   ACCESS MODES   STORAGECLASS    VOLUMEATTRIBUTESCLASS   AGE
local-volume-claim   Pending                                      local-storage   <unset>                 12s


In [ ]:
!kubectl describe pvc local-volume-claim

Name:          local-volume-claim
Namespace:     default
StorageClass:  local-storage
Status:        Pending
Volume:        
Labels:        <none>
Annotations:   <none>
Finalizers:    [kubernetes.io/pvc-protection]
Capacity:      
Access Modes:  
VolumeMode:    Filesystem
Used By:       <none>
Events:
  Type     Reason              Age               From                         Message
  ----     ------              ----              ----                         -------
  Warning  ProvisioningFailed  4s (x2 over 40s)  persistentvolume-controller  storageclass.storage.k8s.io "local-storage" not found


In [9]:
!kubectl apply -f local-volume.yaml

persistentvolume/local-volume unchanged


The PersistentVolumeClaim "local-volume-claim" is invalid: spec: Forbidden: spec is immutable after creation except resources.requests and volumeAttributesClassName for bound claims
  core.PersistentVolumeClaimSpec{
  	AccessModes: {"ReadWriteOnce"},
  	Selector:    nil,
  	Resources: core.VolumeResourceRequirements{
  		Limits: nil,
- 		Requests: core.ResourceList{
- 			s"storage": {i: resource.int64Amount{value: 2147483648}, s: "2Gi", Format: "BinarySI"},
- 		},
+ 		Requests: core.ResourceList{
+ 			s"storage": {i: resource.int64Amount{value: 1073741824}, s: "1Gi", Format: "BinarySI"},
+ 		},
  	},
  	VolumeName:       "",
  	StorageClassName: &"local-storage",
  	... // 4 identical fields
  }



In [10]:
!kubectl delete -f local-volume.yaml

persistentvolume "local-volume" deleted
persistentvolumeclaim "local-volume-claim" deleted


In [11]:
!kubectl apply -f local-volume.yaml

persistentvolume/local-volume created
persistentvolumeclaim/local-volume-claim created


In [12]:
!kubectl get pvc

NAME                 STATUS   VOLUME         CAPACITY   ACCESS MODES   STORAGECLASS    VOLUMEATTRIBUTESCLASS   AGE
local-volume-claim   Bound    local-volume   1Gi        RWO            local-storage   <unset>                 10s


In [13]:
!kubectl describe pvc local-volume-claim

Name:          local-volume-claim
Namespace:     default
StorageClass:  local-storage
Status:        Bound
Volume:        local-volume
Labels:        <none>
Annotations:   pv.kubernetes.io/bind-completed: yes
               pv.kubernetes.io/bound-by-controller: yes
Finalizers:    [kubernetes.io/pvc-protection]
Capacity:      1Gi
Access Modes:  RWO
VolumeMode:    Filesystem
Used By:       <none>
Events:        <none>


In [14]:
!kubectl get pv

NAME           CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                        STORAGECLASS    VOLUMEATTRIBUTESCLASS   REASON   AGE
local-volume   1Gi        RWO            Retain           Bound    default/local-volume-claim   local-storage   <unset>                          77s
